In [1]:
import os
import cv2
import glob
import numpy as np

import matplotlib.pyplot as plt

from microstructure import features, preprocessing, dimension_reduction, classifiers, regression

from sklearn.model_selection import StratifiedKFold

from telegram.utils import send_ok

In [2]:
dir_path = '/Users/mofii/Documents/workplace/microstructure-partII'
data_path = 'data'
classes = ['DUM555', 'DUM560', 'DUM562', 'DUM587', 'DUM588']

In [9]:
haralick = []
hog = []
lbp = []
hu = []
zernike = []
vgg = []
inception = []
Y = []
names = glob.glob(dir_path + '/' + data_path + '/test/*.tif')
haralick = features.haralick_features(names)
hog = features.hog_features(names)
lbp = features.lbp_features(names)
hu = features.hu_moments(names)
zernike = features.zernike_moments(names)
vgg = features.VGG(names)
inception = features.inception(names)
Y += [0] * len(names)
print(haralick.shape)
print(hog.shape)
print(lbp.shape)
print(hu.shape)
print(zernike.shape)
print(vgg.shape)
print(inception.shape)
print(len(Y))

/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/microstructure/features.py:103: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  h, _ = np.histogram(lbp, normed=True, bins=P + 2, range=(0, P + 2))


(13, 13)
(13, 38400)
(13, 12)
(13, 7)
(13, 25)
(13, 1000)
(13, 1000)
13


In [15]:
haralick = []
hog = []
lbp = []
hu = []
zernike = []
vgg = []
inception = []
Y = []
for i in range(len(classes)):
    names = glob.glob(dir_path + '/' + data_path + '/' + classes[i] + '/*.tif')
    if i == 0:
        haralick = features.haralick_features(names)
        hog = features.hog_features(names)
        lbp = features.lbp_features(names)
        hu = features.hu_moments(names)
        zernike = features.zernike_moments(names)
        vgg = features.VGG(names)
        inception = features.inception(names)
    else:
        haralick = np.vstack((haralick, features.haralick_features(names)))
        hog = np.vstack((hog, features.hog_features(names)))
        lbp = np.vstack((lbp, features.lbp_features(names)))
        hu = np.vstack((hu, features.hu_moments(names)))
        zernike = np.vstack((zernike, features.zernike_moments(names)))
        vgg = np.vstack((vgg, features.VGG(names)))
        inception = np.vstack((inception, features.inception(names)))
    Y += [i] * len(names)
Y = np.asarray(Y)

/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/microstructure/features.py:103: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  h, _ = np.histogram(lbp, normed=True, bins=P + 2, range=(0, P + 2))


In [16]:
print(haralick.shape)
print(hog.shape)
print(lbp.shape)
print(hu.shape)
print(zernike.shape)
print(vgg.shape)
print(inception.shape)
print(Y.shape)

(527, 13)
(527, 38400)
(527, 12)
(527, 7)
(527, 25)
(527, 1000)
(527, 1000)
(527,)


In [36]:
skf = StratifiedKFold(n_splits=5)
a = []
f = []
p = []
r = []
rr = []
m = []

In [ ]:
for train_index, test_index in skf.split(np.zeros((len(Y), 1)), Y):
    print(type(train_index[0]))
    train_labels = Y[train_index]
    test_labels = Y[test_index]
    train = vgg[train_index]
    test = vgg[test_index]

    train, test, _ = preprocessing.preprocessing(train, test)

    train, test = dimension_reduction.principal_components(train, test)

    acc, f1, prec, rec, conf, probs, pred = classifiers.svmachine(train, train_labels, test, test_labels)
    a.append(acc)
    f.append(f1)
    p.append(prec)
    r.append(rec)
    
    reg_train_labels = np.random.rand(len(train_labels), 1)
    reg_test_labels = np.random.rand(len(test_labels), 1)
    r2, mse = regression.linear_regression(train, test, reg_train_labels, reg_test_labels)
    rr.append(r2)
    m.append(mse)
send_ok()

<class 'numpy.int64'>


/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


<class 'numpy.int64'>


/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


<class 'numpy.int64'>


/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


<class 'numpy.int64'>


/Users/mofii/anaconda3/envs/microstructure/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


<class 'numpy.int64'>


In [ ]:
print('Accuracy:', np.mean(a))
print('F1 - score:', np.mean(f))
print('Precision:', np.mean(p))
print('Recall:', np.mean(r))

print("R2-score " + str(np.mean(rr)))
print("Mean square error " + str(np.mean(m)))